# SocraticFlanT5 - Improved

## Introduction

In this notebook, we propose a new method to obtain image captions via the Socratic method.
This is an improved pipeline that has for goal to generate similar or improved captions using open-source and free models.

## Imports

In [1]:
# Package loading
from transformers import set_seed
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Local imports
import sys

sys.path.append('..')
try:
    os.chdir('scripts')
except:
    pass
from scripts.coco_captioning_improved import ImageCaptionerImprovedExtended

## Instantiate the improved captioner class

In [2]:
image_captioner = ImageCaptionerImprovedExtended(set_type='demo')

load_places starting!
load_places took 0.0s!
load_objects starting!
load_objects took 0.1s!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Run main

In [ ]:
image_captioner.main()

main starting!
generate_response starting!
generate_response took 77.6s!
generate_response starting!


## Load image and compute image embedding

In [ ]:
image_folder = '../data/images/example_images/'
# Show the image
for img_name in image_captioner.image_manager.demo_names:
    print(f'{img_name}:')
    plt.imshow(image_captioner.image_manager.load_image(image_folder + img_name))
    plt.show()
    cond = image_captioner.generated_caption_df['image_name'] == image_folder + img_name
    generate_caption = image_captioner.generated_caption_df[cond]['generated_caption'].iloc[0]
    print(f'generate caption: "{generate_caption}"\n')
    print('====================================\n')